# Modules

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import emojis

# Preprocessing

In [ ]:
def istime(s):
    pattern = '^([0-9]+)/([0-9]+)/([0-9]+), ([0-9]+):([0-9]+)[ ]?' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

def isNotif(m):
    if len(m.split(": ")) >=2:
        return True
    return False

def get_slice_data(s):
    Line = s.split(' - ') 
    dateTime = Line[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(Line[1:])
    if isNotif(message):
        splitMessage = message.split(':') 
        name = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        name = "Notifications"
        

    return date, time, name, message


In [ ]:
raw_data = open("test.txt","r")
raw_data = raw_data.read().replace("\u202f", " ").split("\n")

lindata= []
buff = ""

for lin in raw_data:
    if istime(lin):
        lindata.append(buff)
        buff = ""
    buff += lin
    #print(istime(lin))

lindata = lindata[1:]
#lindata

In [ ]:
data=dict()
data["Date"] = []
data["Time"] = []
data["Name"] = []
data["Message"] = []

for lin in lindata:
    date, time, name, message = get_slice_data(lin)
    data["Date"].append(date)
    data["Time"].append(time)
    data["Name"].append(name)
    data["Message"].append(message)
#data
df = pd.DataFrame(data,columns=['Date', 'Time', 'Name', 'Message'])
df[100:200]
    

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M %p').dt.time

In [ ]:
people = list(set(data["Name"]))
people.pop(people.index("Notifications"))
people

In [ ]:
x=people
y=[]
for i in x:
    y.append(df[df["Name"] == i].shape[0])


In [ ]:
def show_graphs(x,y):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    ax1.pie(y, labels=x, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Pie Chart')

    ax2.bar(x, y, color=['blue', 'green', 'red', 'purple'])
    ax2.set_title('Bar Chart')

    plt.tight_layout()
    plt.show()


In [ ]:
show_graphs(x,y)

In [ ]:
media_count = dict()
media_raw = list(df[df["Message"]==" <Media omitted>"]["Name"])
for i in people:
    media_count[i]=media_raw.count(i)
media_count

In [ ]:
show_graphs(media_count.keys(),media_count.values())

In [ ]:
word_dict = dict()
for i in df["Message"]:
    for j in i.split(" "):
        if j in word_dict:
            word_dict[j] = word_dict[j] +1
        else:
            word_dict[j] = 1;
sorted_word_dict = sorted(word_dict,key = word_dict.get, reverse=True)
sorted_word_dict

In [ ]:
sorted_word_dict
sorted_word_dict.remove('')
sorted_word_dict.remove('<Media')
sorted_word_dict.remove('omitted>')

In [ ]:
x=[]
y=[]
for i in sorted_word_dict[:10]:
    x.append(i)
    y.append(word_dict[i])


In [ ]:
show_graphs(x,y)